# Sparkify Project

In [1]:
import os
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1636864183959_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Starting a Spark session
spark = SparkSession.builder.appName("Sparkify").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version
-------------------------- -------
beautifulsoup4             4.9.0  
boto                       2.49.0 
jmespath                   0.9.5  
lxml                       4.5.0  
mysqlclient                1.4.2  
nltk                       3.4.5  
nose                       1.3.4  
numpy                      1.16.5 
pip                        9.0.1  
py-dateutil                2.2    
python37-sagemaker-pyspark 1.3.0  
pytz                       2019.3 
PyYAML                     5.3.1  
setuptools                 28.8.0 
six                        1.13.0 
soupsieve                  1.9.5  
wheel                      0.29.0 
windmill                   1.6

In [4]:
sc.install_pypi_package("pillow==8.1.2","https://pypi.org/simple") #Install matplotlib from given PyPI repository

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
sc.install_pypi_package("matplotlib") #Install matplotlib from given PyPI repository

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
sc.install_pypi_package("pandas==0.25.1") #Install pandas version 0.25.1 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version
-------------------------- -------
beautifulsoup4             4.9.0  
boto                       2.49.0 
cycler                     0.11.0 
jmespath                   0.9.5  
kiwisolver                 1.3.2  
lxml                       4.5.0  
matplotlib                 3.4.3  
mysqlclient                1.4.2  
nltk                       3.4.5  
nose                       1.3.4  
numpy                      1.16.5 
pandas                     0.25.1 
Pillow                     8.1.2  
pip                        9.0.1  
py-dateutil                2.2    
pyparsing                  3.0.6  
python-dateutil            2.8.2  
python37-sagemaker-pyspark 1.3.0  
pytz                       2019.3 
PyYAML                     5.3.1  
setuptools                 28.8.0 
six                        1.13.0 
soupsieve                  1.9.5  
wheel                      0.29.0 
windmill                   1.6

In [8]:
# libraries
import pandas as pd
import numpy as np
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.ml.feature import Normalizer, StandardScaler, VectorAssembler
from pyspark.ml.classification import GBTClassifier, LogisticRegression, RandomForestClassifier
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.tuning import CrossValidatorModel as mlm
import re
import datetime
from pyspark.sql import Window
from matplotlib import pyplot as plt
import re
from pyspark.sql import functions as sF
from pyspark.sql import types as sT
from functools import reduce

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Data Cleaning
Handling invalid or missing data like rows without userids or sessionids. 

In [10]:
data_path = 's3n://udacity-dsnd/sparkify/sparkify_event_data.json'
df = spark.read.json(data_path)
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [11]:
df.filter(df.userId == '').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [12]:
# Removing the rows with missing userId
df = df.filter(df.userId != '')
df.filter(df.userId == '').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [13]:
df[df.sessionId == ''].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

## Exploratory Data Analysis (EDA)

In [14]:
df.select("userId").dropDuplicates().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

22278

In [15]:
df.select("page").dropDuplicates().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

22

In [16]:
df.select("sessionId").dropDuplicates().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

228713

In [17]:
# Defining Churn: user who cancelled the subscription
udf_cancelation_event = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())
df = df.withColumn("churn", udf_cancelation_event("page"))
windowval = Window.partitionBy("userId").rangeBetween(Window.unboundedPreceding, Window.unboundedFollowing)
df = df.withColumn("churn", Fsum("churn").over(windowval))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# by spark
df.dropDuplicates(['userId']).groupby(['churn']).count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-----+
|churn|count|
+-----+-----+
|    1| 5003|
|    0|17275|
+-----+-----+

In [19]:
df.dropDuplicates(['userId', 'gender']).groupby(['churn', 'gender']).count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------+-----+
|churn|gender|count|
+-----+------+-----+
|    0|     M| 8995|
|    0|     F| 8279|
|    1|     M| 2656|
|    1|     F| 2347|
|    0|  null|    1|
+-----+------+-----+

In [20]:
df.dropDuplicates(['userId', 'level']).groupby(['churn', 'level']).count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-----+-----+
|churn|level|count|
+-----+-----+-----+
|    0| free|14646|
|    1| paid| 3891|
|    0| paid|12294|
|    1| free| 4147|
+-----+-----+-----+

In [21]:
get_state = udf(lambda x: x.split(',')[1].strip())
df = df.withColumn("state", get_state(df.location))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Feature Engineering

In [22]:
churn_users = df.select('userId', 'churn').dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
print((df.count(), len(df.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(26259199, 20)

In [24]:
def create_dummy_df (col, dictionary):
    col_df = df.select('userId', col).dropDuplicates().replace(dictionary, subset=col)
    col_df = col_df.select('userId', col_df[col].cast('int'))
    print(df.show(5))
    return col_df


gender_df = create_dummy_df('gender', {'M':'1', 'F':'0'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------+------+---------+-------------+---------+--------------------+------+-------------+--------------------+-------+-----+-----+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|location|method|     page| registration|sessionId|                song|status|           ts|           userAgent| userId|churn|state|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------+------+---------+-------------+---------+--------------------+------+-------------+--------------------+-------+-----+-----+
|       Sheena Easton|Logged In|   Nathan|     M|            0|    King|239.62077| paid|Lima, OH|   PUT| NextSong|1538251620000|    16255|Strut (1993 Digit...|   200|1540813195000|"Mozilla/5.0 (Win...|1001393|    0|   OH|
|        Dennis Brown|Logged In|   Nathan|     M|            1|    King|175.38567| paid|Lima, OH|   PUT| NextSon

In [25]:
level_df = create_dummy_df('level', {'paid':'1', 'free':'0'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+-------+-----+-----+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|location|method|    page| registration|sessionId|                song|status|           ts|           userAgent| userId|churn|state|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+-------+-----+-----+
|         Linkin Park|Logged In|   Nathan|     M|            0|    King| 233.1424| paid|Lima, OH|   PUT|NextSong|1538251620000|    10054|Hands Held High (...|   200|1539703092000|"Mozilla/5.0 (Win...|1001393|    0|   OH|
|          The Script|Logged In|   Nathan|     M|            1|    King|260.67546| paid|Lima, OH|   PUT|NextSong|153

In [26]:
song_length = df.filter(df.page=='NextSong').select('userId', 'sessionId', 'length')
song_length = song_length.withColumn('hours', (song_length.length / 3600))
song_length = song_length.groupBy('userId', 'sessionId').sum('hours')
song_length = song_length.groupBy('userId').agg(
                        sF.avg('sum(hours)').alias('mean_hours'), 
                        sF.stddev('sum(hours)').alias('stdev_hours')).na.fill(0)
song_length.toPandas().head(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    userId  mean_hours  stdev_hours
0  1142513    3.418662     3.309651
1  1699838    3.845787     4.136560
2  1380035    5.182219     3.923534
3  1567623    5.363723     4.243871
4  1338783    7.131943     7.885838
5  1390009    4.121716     1.351968
6  1676292   10.293396    11.658975
7  1690101    4.523295     4.794410
8  1695825    0.054037     0.000000
9  1624220    1.905187     2.987784

In [27]:
# Distribution of pages
user_page_distribution = df.groupby('userId').pivot('page').count().na.fill(0) #.toPandas().head(30)
user_page_distribution = user_page_distribution.drop(*['Cancel','Cancellation Confirmation'])
pages_cols = user_page_distribution.columns[1:]
new_df = user_page_distribution.withColumn('total', sum(user_page_distribution[col] for col in pages_cols))
for col in pages_cols:
    new_df = new_df.withColumn(f'norm_{col}', new_df[col] / new_df['total'] * 100.)
new_df = new_df.drop('total')
new_df = new_df.drop(*pages_cols)
oldColumns = new_df.columns
newColumns = ['userId'] + pages_cols
user_page_distribution = reduce(lambda new_df, idx: new_df.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), new_df)
new_df=None
user_page_distribution.toPandas().head(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    userId     About  Add Friend  ...  Thumbs Down  Thumbs Up   Upgrade
0  1338783  0.199800    1.248751  ...     0.799201   3.896104  0.099900
1  1676292  0.152672    2.340967  ...     0.865140   4.122137  0.050891
2  1114507  0.287081    1.818182  ...     1.052632   4.497608  0.095694
3  1624220  0.352113    1.760563  ...     0.704225   4.753521  0.352113
4  1690101  0.138265    1.002420  ...     0.587625   7.500864  0.034566
5  1880560  0.163934    1.693989  ...     1.256831   6.939891  0.163934
6  1567623  0.072727    1.527273  ...     1.090909   4.727273  0.290909
7  1380035  0.208333    1.458333  ...     1.125000   7.541667  0.083333
8  1927371  0.000000    0.418410  ...     1.046025   6.694561  0.209205
9  1732896  0.323625    0.970874  ...     0.809061   4.045307  0.161812

[10 rows x 21 columns]

In [28]:
song_user_df = df.filter(df.page == 'NextSong').groupBy(
                        'userId', 'sessionId').count()
song_user_df = song_user_df.groupBy('userId').agg(
                        sF.avg('count').alias('mean_songs'), 
                        sF.stddev('count').alias('stdev_songs'))
song_user_df = song_user_df.na.fill(0)
song_user_df.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+-----------------+
| userId|       mean_songs|      stdev_songs|
+-------+-----------------+-----------------+
|1624220|          27.6875|43.11646050717366|
|1880560|             99.8|78.61860921108764|
|1567623|75.66666666666667|59.12053869204927|
+-------+-----------------+-----------------+
only showing top 3 rows

In [29]:
artists_user_fans = df.select('userId', 'artist').dropDuplicates().groupBy('userId').count().withColumnRenamed("count", "num_aritst")
artists_user_fans.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+
| userId|num_aritst|
+-------+----------+
|1567623|       867|
|1338783|      1163|
|1271218|      2536|
+-------+----------+
only showing top 3 rows

In [30]:
session_end = df.groupBy('userId', 'sessionId').max('ts').withColumnRenamed(
                                                            'max(ts)', 'end')
session_start = df.groupBy('userId', 'sessionId').min('ts').withColumnRenamed(
                                                            'min(ts)', 'start')
session_df = session_start.join(session_end,['userId', 'sessionId'])
ticks_per_hours = 1000 * 60 * 60
session_df = session_df.select('userId', 'sessionId', ((
    session_df.end-session_df.start)/ticks_per_hours).alias('session_hours'))
session_df.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+------------------+
| userId|sessionId|     session_hours|
+-------+---------+------------------+
|1000025|    42490|1.5402777777777779|
|1000035|    27076| 7.616388888888889|
|1000035|    32236| 4.038333333333333|
+-------+---------+------------------+
only showing top 3 rows

In [31]:
session_user_df = session_df.groupBy('userId').agg(
                        sF.avg('session_hours').alias('mean_session_h'), 
                        sF.stddev('session_hours').alias('stdev_session_h'))
session_user_df = session_user_df.na.fill(0)
session_user_df.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+-----------------+
| userId|   mean_session_h|  stdev_session_h|
+-------+-----------------+-----------------+
|1114507|4.356346153846154|  4.3983561478781|
|1271218|6.910691721132896|7.481044062807567|
|1380035|5.487880658436215|4.504082309398296|
+-------+-----------------+-----------------+
only showing top 3 rows

In [32]:
num_sessions_user_df = df.select('userId', 'sessionId').dropDuplicates().groupby('userId').count().withColumnRenamed('count', 'num_sessions')
num_sessions_user_df.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------+
| userId|num_sessions|
+-------+------------+
|1142513|          10|
|1695825|           1|
|1690101|          37|
+-------+------------+
only showing top 3 rows

In [33]:
def days_since_subscription(df, col_name='days_on'):
    reg_ts = df.select('userId', 'registration').dropDuplicates().withColumnRenamed('registration', 'start')
    end_ts = df.groupBy('userId').max('ts').withColumnRenamed('max(ts)', 'end')
    reg_df = reg_ts.join(end_ts,'userId')
    ticks_per_day = 1000 * 60 * 60 * 24 # as the timestamp is in ticks (0.001 seconds)
    reg_df = reg_df.select('userId', ((reg_df.end-reg_df.start)/ticks_per_day).alias(col_name))
    return reg_df

reg_df = days_since_subscription(df, col_name='days_total_subscription')
reg_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------------+
| userId|days_total_subscription|
+-------+-----------------------+
|1567623|     106.22260416666667|
|1271218|      86.98643518518519|
|1142513|      117.1135300925926|
|1492713|     127.13427083333333|
|1010669|     108.46967592592593|
+-------+-----------------------+
only showing top 5 rows

In [34]:
df_paid = df.filter(df.level=='paid')
paid_df = days_since_subscription(df_paid, col_name='days_paid_subscription')
paid_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------------------+
| userId|days_paid_subscription|
+-------+----------------------+
|1396135|    118.16622685185185|
|1675930|     48.49440972222222|
|1430167|            39.9028125|
|1524082|    58.012442129629626|
|1274556|     80.91873842592592|
+-------+----------------------+
only showing top 5 rows

In [35]:
df_free = df.filter(df.level=='free')
free_df = days_since_subscription(df_free, col_name='days_free_subscription')
free_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------------------+
| userId|days_free_subscription|
+-------+----------------------+
|1114507|     56.02898148148148|
|1271218|     71.42563657407408|
|1492713|    127.13427083333333|
|1440693|     80.96657407407407|
|1883991|     7.657013888888889|
+-------+----------------------+
only showing top 5 rows

In [36]:
paid_free_df = df.filter(df.page=='NextSong').groupBy('userId').pivot('level').count()
paid_free_df = paid_free_df.na.fill(0)
active_cols = paid_free_df.columns[1:]
paid_free_df = paid_free_df.withColumn('total', 
                                       sum(paid_free_df[col] for col in active_cols))
for col in active_cols:
    paid_free_df = paid_free_df.withColumn(f'{col}_percent', 
                                           paid_free_df[col] / paid_free_df.total * 100)
active_cols.append('total')    
paid_free_df = paid_free_df.drop(*active_cols)
paid_free_df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+-----------------+
| userId|      free_percent|     paid_percent|
+-------+------------------+-----------------+
|1732896| 22.37354085603113|77.62645914396887|
|1699838|26.746506986027946|73.25349301397206|
|1676292| 3.536585365853658|96.46341463414633|
|1297189|               0.0|            100.0|
|1816626|               0.0|            100.0|
|1273822| 33.76318874560375|66.23681125439624|
|1674182|             100.0|              0.0|
|1236627|21.168384879725085|78.83161512027492|
|1270219|               0.0|            100.0|
|1716700| 10.18041237113402|89.81958762886599|
+-------+------------------+-----------------+
only showing top 10 rows

In [37]:
paid_free_df = paid_free_df.drop('free_percent')
paid_free_df.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+
| userId|     paid_percent|
+-------+-----------------+
|1972195| 86.8060091443501|
|1115503|54.71264367816092|
|1025974|48.82697947214076|
+-------+-----------------+
only showing top 3 rows

In [38]:
pickedFeatures = [gender_df, paid_free_df, song_length, 
                   user_page_distribution, song_user_df, artists_user_fans, 
                   session_user_df, num_sessions_user_df, reg_df]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
final_df = churn_users
for i, feature in enumerate(pickedFeatures):
    df_to_join = feature
    final_df = final_df.join(df_to_join, 'userId', how='inner')
    final_df = final_df.dropDuplicates()
final_df = final_df.orderBy('userId', ascending=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
final_df.write.format("saved_user_dataset.csv").option("header", "true").csv(
        path = "s3://emrbucket5124/folder", mode = "overwrite"
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
final_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[userId: string, churn: bigint, gender: int, paid_percent: double, mean_hours: double, stdev_hours: double, About: double, Add Friend: double, Add to Playlist: double, Downgrade: double, Error: double, Help: double, Home: double, Login: double, Logout: double, NextSong: double, Register: double, Roll Advert: double, Save Settings: double, Settings: double, Submit Downgrade: double, Submit Registration: double, Submit Upgrade: double, Thumbs Down: double, Thumbs Up: double, Upgrade: double, mean_songs: double, stdev_songs: double, num_aritst: bigint, mean_session_h: double, stdev_session_h: double, num_sessions: bigint, days_total_subscription: double]

## Modeling

In [9]:
final_df = spark.read.option("header","true").csv("s3://emrbucket5124/folder/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
final_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- userId: string (nullable = true)
 |-- churn: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- paid_percent: string (nullable = true)
 |-- mean_hours: string (nullable = true)
 |-- stdev_hours: string (nullable = true)
 |-- About: string (nullable = true)
 |-- Add Friend: string (nullable = true)
 |-- Add to Playlist: string (nullable = true)
 |-- Downgrade: string (nullable = true)
 |-- Error: string (nullable = true)
 |-- Help: string (nullable = true)
 |-- Home: string (nullable = true)
 |-- Login: string (nullable = true)
 |-- Logout: string (nullable = true)
 |-- NextSong: string (nullable = true)
 |-- Register: string (nullable = true)
 |-- Roll Advert: string (nullable = true)
 |-- Save Settings: string (nullable = true)
 |-- Settings: string (nullable = true)
 |-- Submit Downgrade: string (nullable = true)
 |-- Submit Registration: string (nullable = true)
 |-- Submit Upgrade: string (nullable = true)
 |-- Thumbs Down: string (nullable = true)
 |-- T

In [11]:
for col in final_df.columns:
    final_df = final_df.withColumnRenamed(col, col.replace(' ', '_').replace('.', ''))
print(final_df.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['userId', 'churn', 'gender', 'paid_percent', 'mean_hours', 'stdev_hours', 'About', 'Add_Friend', 'Add_to_Playlist', 'Downgrade', 'Error', 'Help', 'Home', 'Login', 'Logout', 'NextSong', 'Register', 'Roll_Advert', 'Save_Settings', 'Settings', 'Submit_Downgrade', 'Submit_Registration', 'Submit_Upgrade', 'Thumbs_Down', 'Thumbs_Up', 'Upgrade', 'mean_songs', 'stdev_songs', 'num_aritst', 'mean_session_h', 'stdev_session_h', 'num_sessions', 'days_total_subscription']

In [12]:
final_df = final_df.withColumn('userId', final_df.userId.cast(sT.IntegerType()))
for col in final_df.columns[1:12]:
    final_df = final_df.withColumn(col, final_df[col].cast(sT.IntegerType()))
for col in final_df.columns[12:]:
    final_df = final_df.withColumn(col, final_df[col].cast(sT.FloatType()))
final_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- userId: integer (nullable = true)
 |-- churn: integer (nullable = true)
 |-- gender: integer (nullable = true)
 |-- paid_percent: integer (nullable = true)
 |-- mean_hours: integer (nullable = true)
 |-- stdev_hours: integer (nullable = true)
 |-- About: integer (nullable = true)
 |-- Add_Friend: integer (nullable = true)
 |-- Add_to_Playlist: integer (nullable = true)
 |-- Downgrade: integer (nullable = true)
 |-- Error: integer (nullable = true)
 |-- Help: integer (nullable = true)
 |-- Home: float (nullable = true)
 |-- Login: float (nullable = true)
 |-- Logout: float (nullable = true)
 |-- NextSong: float (nullable = true)
 |-- Register: float (nullable = true)
 |-- Roll_Advert: float (nullable = true)
 |-- Save_Settings: float (nullable = true)
 |-- Settings: float (nullable = true)
 |-- Submit_Downgrade: float (nullable = true)
 |-- Submit_Registration: float (nullable = true)
 |-- Submit_Upgrade: float (nullable = true)
 |-- Thumbs_Down: float (nullable = true)
 |-- T

In [13]:
final_df = final_df.na.fill(0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
features_labels = final_df.columns[2:]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
features_vector = VectorAssembler(inputCols=final_df.columns[2:], outputCol='features')
input_data = features_vector.transform(final_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
features_scaler = StandardScaler(withMean=True, withStd=True, inputCol='features', outputCol='scaled_features')
features_scaler_fit = features_scaler.fit(input_data)
scaled_inputs = features_scaler_fit.transform(input_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
ml_data = scaled_inputs.select(scaled_inputs.churn.alias('label'), scaled_inputs.scaled_features.alias('features'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
train, test = ml_data.randomSplit([0.80, 0.20], seed=160)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
def get_classifier_metrics(trained_model, train_data, test_data):
    def get_specific_metrics(trained_model, data):
        res2 = trained_model.transform(data).select('label', 'prediction')
        TruePos = res2.filter((res2.prediction==1)& (res2.label == res2.prediction) ).count()
        TrueNeg = res2.filter((res2.prediction==0)& (res2.label == res2.prediction) ).count()
        FalsPos = res2.filter((res2.prediction==1)& (res2.label != res2.prediction) ).count()
        FalsNeg = res2.filter((res2.prediction==0)& (res2.label != res2.prediction) ).count()
        accuracy = res2.filter(res2.label == res2.prediction).count()/res2.count()
        precision = TruePos/(TruePos+FalsPos)
        recall = TruePos/(TruePos+FalsNeg)
        f1score = 2 * precision * recall / (precision + recall)
        return accuracy, precision, recall, f1score
    train_metrics = get_specific_metrics(trained_model, train_data)
    test_metrics = get_specific_metrics(trained_model, test_data)
    labels =['Train', 'Test']
    metrics_names = ['Accuracy', 'Precision', 'Recall', 'F-Score']
    metrics_data =np.array((train_metrics, test_metrics))
    return pd.DataFrame(data=metrics_data.T, columns=labels, index=metrics_names)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### GBT model

In [20]:
model = GBTClassifier()
param_grid = ParamGridBuilder() \
    .addGrid(model.maxDepth,[3, 5, 10]) \
    .addGrid(model.maxBins ,[10, 5]) \
    .addGrid(model.maxIter ,[20, 5]) \
    .build()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
model_evaluator = CrossValidator(estimator=model, estimatorParamMaps=param_grid,
                                      evaluator=MulticlassClassificationEvaluator(),
                                      numFolds=3, seed=100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
fitted_model = model_evaluator.fit(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-22:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 199



In [23]:
get_classifier_metrics(fitted_model, train, test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              Train      Test
Accuracy   0.849899  0.832023
Precision  0.811867  0.774809
Recall     0.424357  0.391892
F-Score    0.557377  0.520513

### Random Forest

In [24]:
model = RandomForestClassifier()
param_grid = ParamGridBuilder() \
    .addGrid(model.maxDepth,[5, 10]) \
    .addGrid(model.impurity,['entropy', 'gini']) \
    .addGrid(model.maxBins ,[10, 5]) \
    .addGrid(model.numTrees ,[20, 10]) \
    .addGrid(model.featureSubsetStrategy ,['sqrt', 'onethird']) \
    .build()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
model_evaluator = CrossValidator(estimator=model, estimatorParamMaps=param_grid,
                                      evaluator=MulticlassClassificationEvaluator(),
                                      numFolds=3, seed=100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
fitted_model = model_evaluator.fit(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-26:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 7776



In [ ]:
get_classifier_metrics(fitted_model, train, test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…